In [217]:
import json
import re

import mysql.connector
import requests
from mysql.connector import IntegrityError


In [321]:
def insert_course(course_info: dict):
    if course_info is None: return
    
    connection = mysql.connector.connect(
        host="coms-309-029.class.las.iastate.edu",
        port=3306,
        user="root",
        password="5b36fb66f5d37314",
        database="Scheduler"
    )
    cursor = connection.cursor()

    sql = (
        "INSERT INTO Course (program_identifier, num, display_name, description, credits, is_variable_credit, is_graded)" +
        "VALUES (%s, %s, %s, %s, %s, %s, %s)"
    )
    values = (
        course_info["program_identifier"],
        course_info["num"],
        course_info["displayName"],
        course_info["description"].encode("latin-1").decode("utf-8"),
        course_info["credits"],
        course_info["is_variable_credit"],
        course_info["is_graded"]
    )
    
    try:
        cursor.execute(sql, values)
    except IntegrityError:
        print(f"Duplicate entry: {course_info['program_identifier']} {course_info['num']}")

    connection.commit()
    cursor.close()
    connection.close()

In [346]:
def scrape_course(program: str, num: int, sem: int = 3) -> tuple[dict, dict] | None:
    url = "https://classes.iastate.edu/app/rest/courses/preferences"
    headers: dict = {"Content-Type": "application/json"}
    payload: str = json.dumps({
        "defSem": sem,
        "selectedTerm": sem,
        "selectedDepartment": program,
        "courseNumber": num
    })

    response: dict = requests.post(url, headers=headers, data=payload).json()
    if len(response["response"]) == 0: return None
    course_data: dict = response["response"][0]
    catalog_data: str = requests.get(course_data["catalogUrl"]).text
    
    # Don't want experimental courses
    if course_data["classNumber"][-1] == "X": return None
    try:
        return {
            "program_identifier": course_data["deptCode"],
            "num": int(re.search(r"([0-9]{1,3})[a-zA-Z]{,3}", course_data["classNumber"]).group(1)),
            "displayName": re.search(r"&#160;[0-9]{1,3}[A-Za-z]{,3}:(.+)</strong>", catalog_data, re.DOTALL).group(1).strip() if len(catalog_data) > 55 else None,
            "description": re.search(r"><br />(.+)</p>", catalog_data, re.DOTALL).group(1).strip() if len(catalog_data) > 55 else None,
            "credits": int(course_data["creditLow"]),
            "is_variable_credit": course_data["creditType"] == 'V',
            "is_graded": course_data["gradeType"] == ""
        }, course_data
    except AttributeError:
        return None


In [ ]:
dept = "STAT"
for n in range(100, 699):
    c = scrape_course(dept, n, sem=3)
    
    if c is None:
        print(f"{dept} {n} doesn't exist")
        continue
    print(f"Inserting {dept} {n}")
    
    insert_course(c[0])


STAT 100 doesn't exist
Inserting STAT 101
STAT 102 doesn't exist
STAT 103 doesn't exist
Inserting STAT 104
STAT 105 doesn't exist
STAT 106 doesn't exist
STAT 107 doesn't exist
STAT 108 doesn't exist
STAT 109 doesn't exist
STAT 110 doesn't exist
STAT 111 doesn't exist
STAT 112 doesn't exist
STAT 113 doesn't exist
STAT 114 doesn't exist
STAT 115 doesn't exist
STAT 116 doesn't exist
STAT 117 doesn't exist
STAT 118 doesn't exist
STAT 119 doesn't exist
STAT 120 doesn't exist
STAT 121 doesn't exist
STAT 122 doesn't exist
STAT 123 doesn't exist
STAT 124 doesn't exist
STAT 125 doesn't exist
STAT 126 doesn't exist
STAT 127 doesn't exist
STAT 128 doesn't exist
STAT 129 doesn't exist
STAT 130 doesn't exist
STAT 131 doesn't exist
STAT 132 doesn't exist
STAT 133 doesn't exist
STAT 134 doesn't exist
STAT 135 doesn't exist
STAT 136 doesn't exist
STAT 137 doesn't exist
STAT 138 doesn't exist
STAT 139 doesn't exist
STAT 140 doesn't exist
STAT 141 doesn't exist
STAT 142 doesn't exist
STAT 143 doesn't ex

In [ ]:
(scrape_course("MATH", 699)[0]["description"].encode("latin-1").decode("utf-8"))

In [330]:
re.search(r"&#160;[0-9]{1,3}[A-Za-z]{,3}:(.+)</strong>", requests.get("https://catalog.iastate.edu/ribbit/?page=getcourse.rjs&code=MATH%20699C&edition=2023-24").text, re.DOTALL)

In [331]:
requests.get("https://catalog.iastate.edu/ribbit/?page=getcourse.rjs&code=MATH%20699C&edition=2023-24").text

'<?xml version="1.0"?>\n<courseinfo>\n</courseinfo>\n'